# 🏦 Credit Limit Review with Human-in-the-Loop

## Overview

This notebook demonstrates **human-in-the-loop (HITL)** workflows for financial services - a pattern where an AI agent proposes credit limit adjustments and a human manager approves or adjusts them.

### 💼 Industry Use Case: Credit Limit Adjustment Workflow

A customer requests a credit limit increase. The workflow:
1. **AI Credit Analyst** proposes a new credit limit based on customer profile
2. **Human Manager** reviews and approves, adjusts, or declines
3. **Iterative refinement** until manager is satisfied

### ⚠️ Important Financial Disclaimer
> **This notebook is for educational purposes only.** Credit limit decisions require full underwriting, compliance review, and regulatory approval. Always follow your institution's policies.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **RequestInfoExecutor** | Pauses workflow for human input |
| **Request/Response Correlation** | Match human replies to specific requests |
| **Structured Agent Output** | Enforce JSON schema with `response_format` |
| **Turn-Based Coordination** | Orchestrate AI ↔ human approval cycles |

### Workflow Architecture

```
Customer Request
    ↓
┌─────────────────────────────────┐
│  AI Analyst Proposes Limit      │
│  (e.g., $8,000 credit limit)    │
└─────────────────────────────────┘
    ↓
CreditManager requests human review
    ↓
┌─────────────────────────────────┐
│  Human Manager Responds:        │
│  • "approve" - accept proposal  │
│  • "higher"  - increase limit   │
│  • "lower"   - decrease limit   │
│  • "decline" - reject request   │
└─────────────────────────────────┘
    ↓
AI adjusts (repeat until approved/declined)
```

## Prerequisites

- ✅ Azure OpenAI Service configured
- ✅ Environment variables: `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- ✅ Azure CLI authentication: Run `az login`

## 1️⃣ Setup and Imports

In [ ]:
import asyncio
from dataclasses import dataclass

import os
from dotenv import load_dotenv
from agent_framework import (
    AgentExecutorRequest,  # Message bundle sent to an AgentExecutor
    AgentExecutorResponse,  # Result returned by an AgentExecutor
    ChatMessage,  # Chat message structure
    Executor,  # Base class for workflow executors
    RequestInfoEvent,  # Event emitted when human input is requested
    Role,  # Enum of chat roles (user, assistant, system)
    WorkflowBuilder,  # Fluent builder for assembling the graph
    WorkflowContext,  # Per run context and event bus
    WorkflowOutputEvent,  # Event emitted when workflow yields output
    WorkflowRunState,  # Enum of workflow run states
    WorkflowStatusEvent,  # Event emitted on run state changes
    handler,  # Decorator to expose an Executor method as a step
    response_handler,  # Decorator for handling human feedback responses
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel

# Load environment variables from .env file
load_dotenv('../../.env')

## 2️⃣ Define Request and Response Models

### ManagerReviewRequest
Request payload sent to human manager for credit limit review via `ctx.request_info()`.

**Fields:**
- `prompt`: Instructions for the manager
- `proposed_limit`: AI's recommended credit limit
- `customer_info`: Customer context for decision

### CreditLimitProposal
Structured output from the AI analyst ensuring reliable parsing.

**Fields:**
- `proposed_limit`: Recommended credit limit amount
- `reasoning`: Brief explanation for the recommendation

In [ ]:
@dataclass
class ManagerReviewRequest:
    """Request payload sent via ctx.request_info() for manager review.
    
    Including the AI's proposal allows the manager UI to display context
    and helps the coordinator track state.
    """
    prompt: str = ""
    proposed_limit: int | None = None
    customer_info: str = ""


class CreditLimitProposal(BaseModel):
    """Structured output from the AI analyst. Enforced via response_format for reliable parsing."""
    proposed_limit: int
    reasoning: str

## 3️⃣ Create CreditManager - The Workflow Coordinator

The `CreditManager` custom executor orchestrates the credit limit review flow using `ctx.request_info()` for human-in-the-loop.

### Handler Methods:

#### 1. `start()`
- **Trigger**: Workflow initialization with customer profile
- **Action**: Send customer data to AI analyst for initial proposal
- **Output**: `AgentExecutorRequest` with customer information

#### 2. `on_analyst_proposal()`
- **Trigger**: Receives `AgentExecutorResponse` from AI analyst
- **Action**: 
  - Parse structured JSON output (`CreditLimitProposal`)
  - Request manager review via `ctx.request_info()`
- **Output**: Pauses workflow for human input

#### 3. `on_manager_decision()` (decorated with `@response_handler`)
- **Trigger**: Receives manager's response to the `request_info` call
- **Action**:
  - Process manager reply ("approve", "higher", "lower", "decline")
  - If "approve": Yield output and complete workflow
  - If guidance: Send feedback to AI for revised proposal
- **Output**: Either workflow output (decision) or next AI request

In [ ]:
class CreditManager(Executor):
    """Coordinates credit limit review between AI analyst and human manager.

    Responsibilities:
    - Kick off AI analysis with customer profile
    - After AI proposal, request manager approval via ctx.request_info()
    - After manager decision, either finalize or ask AI to revise
    """

    def __init__(self, id: str | None = None, customer_info: str = ""):
        super().__init__(id=id or "credit_manager")
        self._customer_info = customer_info
        self._last_proposal: int | None = None

    @handler
    async def start(self, customer_profile: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
        """Start the review by asking the AI analyst for an initial credit limit proposal."""
        self._customer_info = customer_profile
        user = ChatMessage(Role.USER, text=f"Review this customer profile and propose a credit limit:\n\n{customer_profile}")
        await ctx.send_message(AgentExecutorRequest(messages=[user], should_respond=True))

    @handler
    async def on_analyst_proposal(
        self,
        result: AgentExecutorResponse,
        ctx: WorkflowContext,
    ) -> None:
        """Handle the AI analyst's proposal and request manager review via request_info."""
        # Parse structured model output
        text = result.agent_response.text or ""
        try:
            proposal = CreditLimitProposal.model_validate_json(text)
            proposed_limit = proposal.proposed_limit
            reasoning = proposal.reasoning
        except Exception:
            proposed_limit = None
            reasoning = text

        self._last_proposal = proposed_limit

        # Craft manager review prompt
        prompt = (
            f"📊 AI PROPOSAL: ${proposed_limit:,} credit limit\n"
            f"📝 Reasoning: {reasoning}\n\n"
            "Enter your decision:\n"
            "  • approve - Accept this limit\n"
            "  • higher  - AI should propose a higher limit\n"
            "  • lower   - AI should propose a lower limit\n"
            "  • decline - Reject the credit limit request"
        )
        
        # Request human input via request_info (pauses workflow)
        await ctx.request_info(
            request_data=ManagerReviewRequest(
                prompt=prompt, 
                proposed_limit=proposed_limit, 
                customer_info=self._customer_info
            ),
            response_type=str,
        )

    @response_handler
    async def on_manager_decision(
        self,
        original_request: ManagerReviewRequest,
        feedback: str,
        ctx: WorkflowContext[AgentExecutorRequest, str],
    ) -> None:
        """Process manager decision and continue or finalize."""
        reply = feedback.strip().lower()
        last_proposal = original_request.proposed_limit

        if reply == "approve":
            await ctx.yield_output(f"✅ APPROVED: ${last_proposal:,} credit limit")
            return
        
        if reply == "decline":
            await ctx.yield_output("❌ DECLINED: Credit limit increase request rejected")
            return

        # Provide feedback to AI to revise proposal
        user_msg = ChatMessage(
            Role.USER,
            text=(
                f"Manager feedback: propose a {reply} credit limit. "
                f"Previous proposal was ${last_proposal:,}. "
                f'Return ONLY JSON: {{"proposed_limit": <int>, "reasoning": "<brief explanation>"}}'
            ),
        )
        await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))

## 4️⃣ Understanding ctx.request_info()

### How Human-in-the-Loop Works:

The `ctx.request_info()` method provides a **workflow-native bridge** for human input:

1. **Executor calls** `ctx.request_info(request_data=..., response_type=str)`
2. **Workflow pauses** and emits `RequestInfoEvent` with typed payload
3. **Application collects** human input and calls `workflow.send_responses_streaming()`
4. **Workflow resumes** and delivers response to `@response_handler` method

### FSI Benefits:

| Benefit | Description |
|---------|-------------|
| **Compliance** | Ensures human oversight for credit decisions |
| **Audit Trail** | All approvals correlated via request_id |
| **Flexibility** | Manager can approve, adjust, or escalate |
| **Type Safety** | Structured request/response with correlation |

## 5️⃣ Build the Workflow Graph

### Components:

1. **CreditManager**: Start executor and coordinator (handles `request_info` internally)
2. **Credit Analyst Agent**: AI agent wrapped via `register_agent()`

### Edges (Message Flow):

```
CreditManager → CreditAnalyst (request proposal)
CreditAnalyst → CreditManager (return proposal)
                    ↓
         ctx.request_info() pauses workflow
                    ↓
         Human manager provides response
                    ↓
         @response_handler resumes workflow
```

In [ ]:
# Create the AI Credit Analyst agent factory
def create_credit_analyst():
    """Factory function that creates the Credit Analyst agent."""
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"))
    
    return AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    ).as_agent(
        name="CreditAnalyst",
        instructions=(
            "You are a Credit Analyst at a retail bank. "
            "Based on customer profiles, you propose appropriate credit limits. "
            "Consider factors like income, credit score, and existing debt. "
            "Typical limits range from $1,000 to $25,000. "
            'You MUST return ONLY a JSON object exactly matching this schema: '
            '{"proposed_limit": <integer>, "reasoning": "<brief explanation>"}. '
            "No additional text."
        ),
        default_options={"response_format": CreditLimitProposal},
    )

print("✅ Credit Analyst agent factory created")

# Build the workflow graph using register pattern
workflow = (
    WorkflowBuilder()
    .register_agent(create_credit_analyst, name="credit_analyst")
    .register_executor(lambda: CreditManager(id="credit_manager"), name="credit_manager")
    .set_start_executor("credit_manager")
    .add_edge("credit_manager", "credit_analyst")  # Ask AI analyst for credit limit proposal
    .add_edge("credit_analyst", "credit_manager")  # AI's proposal comes back to coordinator
).build()

print("✅ Credit limit review workflow built")
print("\nWorkflow Graph:")
print("  CreditManager (start) → CreditAnalyst (AI)")
print("  CreditAnalyst → CreditManager")
print("  CreditManager uses ctx.request_info() for human approval")

## 6️⃣ Run the Credit Limit Review

### Customer Profile

We'll submit a credit limit increase request for review:
- **Customer**: Sarah Chen
- **Current Limit**: $5,000
- **Requested Increase**: $10,000
- **Profile**: Strong income, good credit history

### ⚠️ IMPORTANT: How to Respond to the Input Dialog

When the workflow pauses for your decision, **an input box will appear at the top of VS Code**.

> **You MUST type your response in the input box, then press Enter.**  
> Just pressing Enter without typing anything will NOT work.

### Valid Responses (type one of these):

| Type This | What It Does |
|-----------|--------------|
| `approve` | Accept the AI's proposed credit limit |
| `higher` | Ask AI to propose a higher limit |
| `lower` | Ask AI to propose a lower limit |
| `decline` | Reject the credit limit request |
| `exit` | Cancel the workflow |

In [ ]:
async def run_credit_limit_review():
    """Run the interactive credit limit review workflow."""
    
    # Customer profile for credit limit review
    customer_profile = """
    CREDIT LIMIT INCREASE REQUEST
    =============================
    Customer: Sarah Chen
    Account: **** 4521
    Current Credit Limit: $5,000
    Requested New Limit: $10,000
    
    CUSTOMER PROFILE:
    - Customer Since: 2019
    - Payment History: 100% on-time (5 years)
    - Current Utilization: 35%
    - Annual Income: $95,000
    - Credit Score: 760
    - Existing Debt: $12,000 (auto loan)
    - Employment: Senior Analyst, Tech Corp (6 years)
    
    ACCOUNT HISTORY:
    - No late payments
    - Average monthly spend: $1,800
    - Previous limit increase: 2022 ($3,000 → $5,000)
    """
    
    # Workflow state
    pending_responses: dict[str, str] | None = None
    completed = False
    workflow_output: str | None = None

    # Display header
    print("=" * 70)
    print("🏦 CREDIT LIMIT REVIEW WORKFLOW")
    print("=" * 70)
    print(customer_profile)
    print("=" * 70)
    print("\n👨‍💼 MANAGER INSTRUCTIONS:")
    print("  • approve - Accept the proposed credit limit")
    print("  • higher  - Request a higher limit proposal")
    print("  • lower   - Request a lower limit proposal")
    print("  • decline - Reject the credit limit request")
    print("=" * 70 + "\n")

    while not completed:
        # First iteration uses run_stream with customer profile
        # Subsequent iterations use send_responses_streaming
        stream = (
            workflow.send_responses_streaming(pending_responses) 
            if pending_responses 
            else workflow.run_stream(customer_profile)
        )
        
        # Collect events for this turn
        events = [event async for event in stream]
        pending_responses = None

        # Process events
        requests: list[tuple[str, str]] = []
        for event in events:
            if isinstance(event, RequestInfoEvent) and isinstance(event.data, ManagerReviewRequest):
                requests.append((event.request_id, event.data.prompt))
            elif isinstance(event, WorkflowOutputEvent):
                workflow_output = str(event.data)
                completed = True

        # Check if workflow is paused waiting for manager input
        idle_with_requests = any(
            isinstance(e, WorkflowStatusEvent) and e.state == WorkflowRunState.IDLE_WITH_PENDING_REQUESTS
            for e in events
        )
        
        if idle_with_requests:
            print("\n⏳ Awaiting manager decision...")

        # If completed, show final result
        if completed:
            print("\n" + "=" * 70)
            print("📋 FINAL DECISION")
            print("=" * 70)
            print(workflow_output)
            print("=" * 70)
            break

        # Collect manager input for each request
        if requests:
            pending_responses = {}
            for request_id, prompt in requests:
                print("\n" + "-" * 50)
                print(prompt)
                print("-" * 50)
                user_input = input("👨‍💼 Manager Decision: ").strip()
                
                # Handle exit
                if user_input.lower() == "exit":
                    print("\n❌ Review cancelled by manager")
                    return
                    
                pending_responses[request_id] = user_input

    print("\n✅ Credit limit review workflow complete!")
    print("\n⚠️ DISCLAIMER: This is a demonstration only. Actual credit")
    print("   decisions require full underwriting and compliance review.")

## 7️⃣ Run the Credit Limit Review

In [ ]:
await run_credit_limit_review()

## 📝 Key Takeaways

### Human-in-the-Loop for FSI

| Benefit | Description |
|---------|-------------|
| **Compliance** | Human oversight required for credit decisions |
| **Accountability** | Clear audit trail of approval decisions |
| **Flexibility** | Manager can adjust AI recommendations |
| **Risk Management** | Human judgment for edge cases |

### Industry Use Cases for Human-in-the-Loop

- **Credit Decisions**: Limit increases, new accounts
- **Fraud Escalation**: Suspicious transaction review
- **Compliance Gates**: Regulatory approval workflows
- **Exception Handling**: Edge case decisions
- **High-Value Transactions**: Large transfer approvals

